In [1]:
print('HELLO')

HELLO


In [2]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninsta

In [3]:
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.5
    Uninstalling spacy-3.8.5:
      Successfully uninstalled spacy-3.8.5


In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [5]:
!nvidia-smi

Mon May 19 12:20:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
cv_data = json.load(open("/kaggle/input/resumedataset/train_data.json"))

In [7]:
!python -m spacy init fill-config /kaggle/input/resumedataset/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot["entities"]
        ents = []
        entity_indices = []
        for start, end, label in annot:
          skip_entity = False
          for idx in range(start, end):
            if idx in entity_indices:
              print('TRUE',idx)
              skip_entity = True
              break
          if skip_entity == True:
            continue
          entity_indices = entity_indices + list(range(start, end))
          try:
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
          except:
            continue
          if span is None:
            err_data = str([start, end]) + " " + str(text) + "\n"
            file.write(err_data)
          else:
            ents.append(span)
        try:
          doc.ents = ents
          db.add(doc)
        except:
          pass
    return db

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
len(train), len(test)

(140, 60)

In [11]:
file = open("error_data.txt", "w")
db = get_spacy_doc(file, train)
db.to_disk("train_data.spacy")

db = get_spacy_doc(file, test)
db.to_disk("test_data.spacy")

file.close()

  5%|▌         | 7/140 [00:00<00:01, 67.43it/s]

TRUE 549
TRUE 26
TRUE 4708
TRUE 15
TRUE 34


 26%|██▋       | 37/140 [00:00<00:01, 93.86it/s]

TRUE 1258
TRUE 497
TRUE 370
TRUE 13
TRUE 3461
TRUE 2528
TRUE 2076
TRUE 1865
TRUE 3385
TRUE 10
TRUE 2886
TRUE 1161
TRUE 3535
TRUE 15
TRUE 381
TRUE 6861


 34%|███▎      | 47/140 [00:00<00:01, 84.26it/s]

...

TRUE 1417
TRUE 1209
TRUE 1342
TRUE 4231
TRUE 7777
TRUE 458
TRUE 3981
TRUE 3803
TRUE 46
TRUE 983
TRUE 67
TRUE 3


 63%|██████▎   | 38/60 [00:00<00:00, 93.45it/s]

TRUE 4374
TRUE 3132
TRUE 2665
TRUE 2562
TRUE 2407
TRUE 2184
TRUE 556
TRUE 263
TRUE 60
TRUE 13883
TRUE 11578
TRUE 3243
TRUE 667
TRUE 2733
TRUE 3939
TRUE 2907


100%|██████████| 60/60 [00:00<00:00, 92.24it/s]

TRUE 8134
TRUE 4186
TRUE 4774
TRUE 941


In [12]:
!python -m spacy train /kaggle/working/config.cfg \
--output /kaggle/working/output \
--paths.train /kaggle/working/train_data.spacy \
--paths.dev /kaggle/working/test_data.spacy \
--gpu-id 0


✔ Created output directory: /kaggle/working/output
ℹ Saving to output directory: /kaggle/working/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-05-19 12:21:24,417] [INFO] Set up nlp object from config
[2025-05-19 12:21:24,437] [INFO] Pipeline: ['transformer', 'ner']
[2025-05-19 12:21:24,440] [INFO] Created vocabulary
[2025-05-19 12:21:24,440] [INFO] Finished initializing nlp object
tokenizer_config.json: 100%|██████████████████| 25.0/25.0 [00:00<00:00, 178kB/s]
config.json: 100%|█████████████████████████████| 481/481 [00:00<00:00, 3.37MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 19.5MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 27.5MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 29.1MB/s]
2025-05-19 12:21:33.712729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register fa